## SDR data processing

Process all collected SDR data and generate saved files with matched filter outputs. We assume the folder contains only files with the correct filenames

In [10]:
import os
import datetime
import mixing_tools as mtools
import numpy as np

# create a list of all directories with data
basedir = "/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data"
outdir = "/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter"  # ALT output

devices = ["eggtart", "bananacream"]  # list of devices

# spreading factors used for the experiments
sf = {"outdoor_11june":9, "outdoor_13june":10}
# which recieve frequencies were used, based on filename
frxdict = {"433.750M":433.75e6, "904.750M":904.75e6}

exps = list(sf.keys())  # list of experiments
datadirs = []

for exp in exps:
    for dev in devices:
        d = os.path.join(basedir, exp, dev)
        dout = os.path.join(outdir, exp, dev)
        datadirs.append((d, dout, sf[exp]))

        
def createOutputFilename(inputfile, outdir=None):
    if outdir:
        dirname=outdir
    else:
        dirname = os.path.dirname(inputfile)
    resfilename = "mf_" + os.path.basename(inputfile)
    return os.path.join(dirname, resfilename)


def loraSymboltime(bw, sf):
    return (2**sf)/bw


def processFile(datafile, outputfile, Ts, t0, bwChirp, fctx, fcrx, sf):
    # load data
    data = mtools.load_IQBinary_int16(datafile)
    dataBB, fcBB, TsBB = mtools.basebandShift(data, Ts, bwSignal=bwChirp,
                        fcSignal=fctx, fcData=fcrx, signalType='ds')

    # construct template and filter
    TChirp = loraSymboltime(bwChirp, sf)
    fcBBChirp = fctx - fcBB
    # print("freq offset of chirp in baseband: ", fcBBChirp)
    template = mtools.chirpTemplate(Ts=TsBB, fc=fcBBChirp, bw=bwChirp, TChirp=TChirp, direction='down')
    # print("size of chirp template: ", template.size)
    result = mtools.matchedFilter(dataBB, template)

    # save results
    np.savez(outputfile, data=result, Ts=TsBB, t0=t0, fc=fcBB)

    
def filenameParse(fullpath):
    filename = os.path.basename(fullpath)
    _, freqinfo, dateinfo = filename.split("_", 2)
    dtstr = dateinfo.split(".")[0]
    dtobj = datetime.datetime.strptime(dtstr, "%Y-%m-%d_%H-%M-%S")
    dtobjutc = dtobj.replace(tzinfo=datetime.timezone.utc)
    t0 = dtobjutc.timestamp()
    frx = frxdict[freqinfo]
    ftx = ftxdict[freqinfo]
    return frx, ftx, t0


# objective: process all files in the provided directory
def processDataDirectory(directory, dirOut, sf):
    filenames = os.listdir(directory)
    for filename in filenames:
        fullpath = os.path.join(directory, filename)
        fcrx, fctx, t0 = filenameParse(fullpath)
        outfile = createOutputFilename(fullpath, dirOut)
        print("{} -> {}: fcRX {:.2f}, fcTX {:.2f}, t0 {}".format(fullpath, outfile, fcrx/1e6, fctx/1e6, t0))
        processFile(datafile=fullpath,
                    outputfile=outfile,
                    Ts=1e-6,
                    t0=t0,
                    bwChirp=125e3,
                    fctx=fctx,
                    fcrx=fcrx,
                    sf=sf)


os.makedirs(outdir, exist_ok=True)
        
for d, dout, sf in datadirs:
    print("\n### processing {} at SF {} ###".format(d, sf))
    os.makedirs(dout, exist_ok=True)
    processDataDirectory(d, dout, sf)
    print("######")



### processing /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/eggtart at SF 9 ###
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/eggtart/exp1_433.750M_2020-06-11_17-57-37.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/eggtart/mf_exp1_433.750M_2020-06-11_17-57-37.000.dat: fcRX 433.75, fcTX 434.00, t0 1591898257.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/eggtart/exp1_433.750M_2020-06-11_18-05-42.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/eggtart/mf_exp1_433.750M_2020-06-11_18-05-42.000.dat: fcRX 433.75, fcTX 434.00, t0 1591898742.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/eggtart/exp1_433.750M_2020-06-11_18-11-18.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/eggtart/mf_exp1_433.750M_2020-06-11_18-11-18.000.dat: fcRX 433.75, fcTX 434

/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/eggtart/exp1_904.750M_2020-06-11_18-39-31.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/eggtart/mf_exp1_904.750M_2020-06-11_18-39-31.000.dat: fcRX 904.75, fcTX 904.99, t0 1591900771.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/eggtart/exp1_904.750M_2020-06-11_18-40-48.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/eggtart/mf_exp1_904.750M_2020-06-11_18-40-48.000.dat: fcRX 904.75, fcTX 904.99, t0 1591900848.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/eggtart/exp1_904.750M_2020-06-11_18-42-13.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/eggtart/mf_exp1_904.750M_2020-06-11_18-42-13.000.dat: fcRX 904.75, fcTX 904.99, t0 1591900933.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/eggtart/exp1_90

/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/bananacream/exp1_433.750M_2020-06-11_18-40-58.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/bananacream/mf_exp1_433.750M_2020-06-11_18-40-58.000.dat: fcRX 433.75, fcTX 434.00, t0 1591900858.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/bananacream/exp1_433.750M_2020-06-11_18-42-23.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/bananacream/mf_exp1_433.750M_2020-06-11_18-42-23.000.dat: fcRX 433.75, fcTX 434.00, t0 1591900943.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/bananacream/exp1_433.750M_2020-06-11_18-43-59.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/bananacream/mf_exp1_433.750M_2020-06-11_18-43-59.000.dat: fcRX 433.75, fcTX 434.00, t0 1591901039.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoo

/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/bananacream/exp1_904.750M_2020-06-11_18-46-21.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/bananacream/mf_exp1_904.750M_2020-06-11_18-46-21.000.dat: fcRX 904.75, fcTX 904.99, t0 1591901181.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/bananacream/exp1_904.750M_2020-06-11_18-52-30.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/bananacream/mf_exp1_904.750M_2020-06-11_18-52-30.000.dat: fcRX 904.75, fcTX 904.99, t0 1591901550.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/bananacream/exp1_904.750M_2020-06-11_18-54-06.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_11june/bananacream/mf_exp1_904.750M_2020-06-11_18-54-06.000.dat: fcRX 904.75, fcTX 904.99, t0 1591901646.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoo

/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/eggtart/exp1_433.750M_2020-06-13_17-43-54.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/eggtart/mf_exp1_433.750M_2020-06-13_17-43-54.000.dat: fcRX 433.75, fcTX 434.00, t0 1592070234.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/eggtart/exp1_433.750M_2020-06-13_17-44-52.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/eggtart/mf_exp1_433.750M_2020-06-13_17-44-52.000.dat: fcRX 433.75, fcTX 434.00, t0 1592070292.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/eggtart/exp1_433.750M_2020-06-13_17-45-40.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/eggtart/mf_exp1_433.750M_2020-06-13_17-45-40.000.dat: fcRX 433.75, fcTX 434.00, t0 1592070340.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/eggtart/exp1_43

/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/eggtart/exp1_904.750M_2020-06-13_18-02-36.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/eggtart/mf_exp1_904.750M_2020-06-13_18-02-36.000.dat: fcRX 904.75, fcTX 904.99, t0 1592071356.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/eggtart/exp1_433.750M_2020-06-13_17-51-00.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/eggtart/mf_exp1_433.750M_2020-06-13_17-51-00.000.dat: fcRX 433.75, fcTX 434.00, t0 1592070660.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/eggtart/exp1_433.750M_2020-06-13_17-54-14.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/eggtart/mf_exp1_433.750M_2020-06-13_17-54-14.000.dat: fcRX 433.75, fcTX 434.00, t0 1592070854.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/eggtart/exp1_43

/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/bananacream/exp1_433.750M_2020-06-13_17-43-54.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/bananacream/mf_exp1_433.750M_2020-06-13_17-43-54.000.dat: fcRX 433.75, fcTX 434.00, t0 1592070234.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/bananacream/exp1_433.750M_2020-06-13_17-44-52.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/bananacream/mf_exp1_433.750M_2020-06-13_17-44-52.000.dat: fcRX 433.75, fcTX 434.00, t0 1592070292.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/bananacream/exp1_433.750M_2020-06-13_17-45-40.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/bananacream/mf_exp1_433.750M_2020-06-13_17-45-40.000.dat: fcRX 433.75, fcTX 434.00, t0 1592070340.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoo

/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/bananacream/exp1_904.750M_2020-06-13_18-02-36.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/bananacream/mf_exp1_904.750M_2020-06-13_18-02-36.000.dat: fcRX 904.75, fcTX 904.99, t0 1592071356.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/bananacream/exp1_433.750M_2020-06-13_17-50-12.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/bananacream/mf_exp1_433.750M_2020-06-13_17-50-12.000.dat: fcRX 433.75, fcTX 434.00, t0 1592070612.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/bananacream/exp1_433.750M_2020-06-13_18-04-22.000.dat -> /media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/matchedfilter_n10ppm/outdoor_13june/bananacream/mf_exp1_433.750M_2020-06-13_18-04-22.000.dat: fcRX 433.75, fcTX 434.00, t0 1592071462.0
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoo